# 5 - Process spectrogram data

Loads adjusted data and formats for spectral analysis.

## Imports

Necessary modules for analysis.

In [1]:
# import modules

import xarray as xr
import numpy as np
import scipy.signal as sig
for i in range(2):
    %matplotlib notebook

In [51]:
# import data

adcp = 'Slope'     # Slope(2013,2014,2017,2018), Axis75(2013,2014), Axis55(2017,2018)
year = 2018
ds_in = xr.open_dataset(f'../Data/data/adj/adj_{adcp}_{year}_0.nc')

n_seg = ds_in.n_seg
if n_seg > 1:
    ds = [ds_in]
    for i in range(n_seg):
        if i > 0:
            ds_temp = xr.open_dataset(f'../Data/data/adj/adj_{adcp}_{year}_{i}.nc')
            ds.append(ds_temp)
elif n_seg == 1:
    ds = [ds_in]
    
# print(ds)

In [52]:
# extract variables

t_stamp = ds[0].t_stamp
depth = ds[0].depth.values
d = ds[0].d
start_date = ds[0].start_date
end_date = ds[0].end_date

## Format spectrogram data
Removes the mean from each depth. Performs spectrogram process with optimised parameters for visual analysis of spectra. Calculates 95% confidence intervals using a chi$^2$ method, WKB scaling is applied, and the noise floor is shown.

In [53]:
# spectra data adjustments & Welch parameters

time_total = 0
for i in range(n_seg):
    time_total += len(ds[i].t_seg)
print('Total time length:',time_total)

# set Welch parameters

fs = 1.11e-3                  # 4 samples per HOUR, 1.11e-3 per SECOND
win = 'hann'                  # optimal window for averaging
nps = 1024                # find optimal window for nperseg (1024 ~10 days)
overlap = nps // 2            # 50% overlap, default   

# remove short segments

t_short = []
for i in range(n_seg):
    if len(ds[i].t_seg) < nps:
        t_short.append(i)
for i in sorted(t_short, reverse=True):
    del ds[i]
n_seg = n_seg - len(t_short)

time_total = 0
for i in range(n_seg):
    time_total += len(ds[i].t_seg)
print(len(t_short),'segment(s) too short, new total time length:',time_total)

Total time length: 31363
0 segment(s) too short, new total time length: 31363


In [54]:
# remove mean at each depth

um,vm = [],[]
for i in range(n_seg):
    um_seg,vm_seg = [],[]
    uorig_temp = ds[i].uorig.values
    vorig_temp = ds[i].vorig.values
    for j in range(d):
        umtemp = np.copy(uorig_temp[:,j]) - np.nanmean(uorig_temp[:,j])
        vmtemp = np.copy(vorig_temp[:,j]) - np.nanmean(vorig_temp[:,j])
        um_seg.append(umtemp)
        vm_seg.append(vmtemp)
    um.append(um_seg)     # list[segment][depth][time]
    vm.append(vm_seg)     # 0 is upper depth index

In [55]:
# spectrograms at each depth (PSD at each depth and in time steps of about 10 days)

spect,specf,Sxu,Sxv,Sxxu,Sxxv = [],[],[],[],[],[]
for i in range(n_seg): 
    spect_temp,specf_temp,Sxu_temp,Sxv_temp,Sxxu_temp,Sxxv_temp = [],[],[],[],[],[]
    for z in range(d):
        # get spectrogram    
        u_f, u_t, u_Sxx = sig.spectrogram(um[i][z], fs=fs, window=win, \
                                                            nperseg = nps, noverlap = overlap, return_onesided=True)
        v_f, v_t, v_Sxx = sig.spectrogram(vm[i][z], fs=fs, window=win, \
                                                            nperseg = nps, noverlap = overlap, return_onesided=True)

        # 'whiten' spectrogram (flatten it) by multiplying Sxx by frequency**2
        Sxx_u_white = np.copy(u_Sxx)
        for j in range(len(u_t)):
            Sxx_u_white[:,j] = Sxx_u_white[:,j]*(u_f**2)
        Sxx_v_white = np.copy(v_Sxx)
        for j in range(len(v_t)):
            Sxx_v_white[:,j] = Sxx_v_white[:,j]*(v_f**2)

        # convert spectro_t to datetime for x-axis on plots
        spectro_t4 = u_t*fs
        spectro_time_len = len(spectro_t4)
        spectro_time_axis = np.zeros([spectro_time_len],dtype='datetime64[s]')
        for k in range(spectro_time_len):
            j = np.int(spectro_t4[k])
            spectro_time_axis[k] = ds[i].t_seg[j].values

        spect_temp.append(spectro_time_axis)
        specf_temp.append(u_f)
        Sxu_temp.append(u_Sxx)    # non-whitened
        Sxv_temp.append(v_Sxx)
        Sxxu_temp.append(Sxx_u_white)  # whitened
        Sxxv_temp.append(Sxx_v_white)
        
    spect.append(spect_temp[0])   # append arrays to lists (same times and frequencies at each depth)
    specf.append(specf_temp[0])
    Sxu.append(Sxu_temp)          # non-whitened
    Sxv.append(Sxv_temp)
    Sxxu.append(Sxxu_temp)        # list[array_segment][depth][frequency][time_segment]
    Sxxv.append(Sxxv_temp)

In [56]:
# apply WKB scaling at each depth

scaling_array = np.load('../project/archive/N2/scaling_array.npy')
GM_depths = scaling_array[0]                                 # depths range from -4 to -924 metres
GM_scale = scaling_array[1]
int_scale = np.interp(depth,-GM_depths,GM_scale)

Sxu_WKB,Sxv_WKB,Sxxu_WKB,Sxxv_WKB = [],[],[],[]

for i in range(n_seg):
    Sxu_WKB_temp,Sxv_WKB_temp,Sxxu_WKB_temp,Sxxv_WKB_temp = [],[],[],[]
    for j in range(d):                                       # values go in descending depth (0 is upper index)
        Sxu_WKB_temp.append(Sxu[i][j]/int_scale[j])          # list[depth][frequency]
        Sxv_WKB_temp.append(Sxv[i][j]/int_scale[j])
        Sxxu_WKB_temp.append(Sxxu[i][j]/int_scale[j])        # list[depth][frequency]
        Sxxv_WKB_temp.append(Sxxv[i][j]/int_scale[j])
    Sxu_WKB.append(Sxu_WKB_temp)
    Sxv_WKB.append(Sxv_WKB_temp)
    Sxxu_WKB.append(Sxxu_WKB_temp)
    Sxxv_WKB.append(Sxxv_WKB_temp)

## Save
Save key values and arrays to NetCDF format using xarray.

In [57]:
# save to .nc files

for i in range(n_seg):
    ds_out = xr.Dataset( 
        data_vars=dict(
            Sxxu=(['depth','specf','spect'], Sxu_WKB[i]),                   # depth-freq-time data for power at each depth
            Sxxv=(['depth','specf','spect'], Sxv_WKB[i]),
            Sxxu_white=(['depth','specf','spect'], Sxxu_WKB[i]),         # whitened depth-freq-time data for power at each depth
            Sxxv_white=(['depth','specf','spect'], Sxxv_WKB[i]),
        ),
        coords=dict(
            depth=depth,                               # depth values
            specf=specf[i],                            # spectrogram frequency bins
            spect=spect[i],                            # spectrogram time bins
        ),
        attrs=dict(
            description=f'Depth-freq-time data for {adcp} {t_stamp} segment {i} (from 0 to {n_seg - 1}).',
            n_seg=n_seg,                               # n_seg for major datagaps each year
            adcp=adcp,                                 # adcp
            t_stamp=t_stamp,                           # time stamp
            t=time_total,                              # length of initial time series
            d=d,                                       # depth range
            start_date=str(ds[i].t_seg[0].values),     # start date of initial time series segment
            end_date=str(ds[i].t_seg[-1].values),      # end date of initial time series segment
            fs=fs,                                     # sampling frequency
            window=win,                                # fft window
            nps=nps,                                   # fft segment length
            overlap=overlap,                           # fft window overlap
        ),
    ) 

    ds_out.to_netcdf(f'../Data/data/spectro/spectro_{adcp}_{t_stamp}_{i}.nc')